In [91]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [132]:
hospitals = pd.read_excel('/Users/amanchopra/Documents/Work/Aarete/Provider Transparency Project/Hospitals.xlsx', sheet_name='Hospitals')
hospitals.info()
hospitals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7597 entries, 0 to 7596
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   X                                             7596 non-null   float64       
 1   Y                                             7596 non-null   float64       
 2   OBJECTID                                      7596 non-null   float64       
 3   ID                                            7596 non-null   float64       
 4   NAME                                          7597 non-null   object        
 5   ADDRESS                                       7597 non-null   object        
 6   CITY                                          7597 non-null   object        
 7   STATE                                         7597 non-null   object        
 8   ZIP                                           7597 non-null   int64 

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,Downloadable Data,Type,Price URL,ChargeMaster,Hospital Standard Charges,Average Charges billed per DRg,Hospital Price Transparency charge info,Hospital price transparency case information,COVID Test,Shoppable Services
0,-1.331889e+07,4.346975e+06,1.0,5793230.0,CENTRAL VALLEY GENERAL HOSPITAL,1025 NORTH DOUTY STREET,HANFORD,CA,93230,NOT AVAILABLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.322651e+07,4.049626e+06,2.0,53391362.0,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,150 VIA MERIDA,WESTLAKE VILAGE,CA,91362,NOT AVAILABLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.315620e+07,4.031978e+06,3.0,11190023.0,EAST LOS ANGELES DOCTORS HOSPITAL,4060 WHITTIER BOULEVARD,LOS ANGELES,CA,90023,NOT AVAILABLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.317190e+07,4.041752e+06,4.0,17090028.0,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,6245 DE LONGPRE AVENUE,HOLLYWOOD,CA,90028,NOT AVAILABLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.313208e+07,4.037270e+06,5.0,23691706.0,KINDRED HOSPITAL BALDWIN PARK,14148 FRANCISQUITO AVENUE,BALDWIN PARK,CA,91706,NOT AVAILABLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
def get_data_for_provider(driver, prov_name, prov_zip_code):
    i = 1
    data = []

    while True:   
        prov_name_url = '-'.join(prov_name.lower().split(' '))
        driver.get(f'https://turquoise.health/service_offerings?q=&service_name=&location={prov_zip_code}&provider_name={prov_name_url}&page={i}&distance=10')


        prices_div = driver.find_elements_by_xpath('//div[contains(@class, "browse-services estimated-service-price servic-location-cont")]')

        if not prices_div:
            break

        prices = prices_div[0].find_element_by_tag_name('ul').find_elements_by_xpath('//div[contains(@class, "service-info-cont")]')
        #time.sleep()

        for price in prices:
            info = price.text.split('\n')
            row = [prov_name, prov_zip_code] + info[:3]
            data.append(row)

        i += 1

    prov_data = pd.DataFrame(data, columns=['Provider', 'Zip Code', 'CPT', 'Service', 'Cash Price'])
    
    return prov_data
        

In [140]:
pricing_df = pd.DataFrame()

chrome_options = Options()
chrome_options.add_argument('--incognito')

driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)

for ind, row in hospitals.iterrows():
    if ind == 5:
        break
    prov_data = get_data_for_provider(driver, row['NAME'], row['ZIP'])
    pricing_df = pd.concat([pricing_df, prov_data]).reset_index(drop=True)

driver.quit()
pricing_df



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/amanchopra/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


,Provider,Zip Code,CPT,Service,Cash Price
0,CENTRAL VALLEY GENERAL HOSPITAL,93230,HCPCS 10061,Drainage Of Skin Abscess,$143.54
1,CENTRAL VALLEY GENERAL HOSPITAL,93230,MSDRG 101,Seizures,"$8,284.88"
2,CENTRAL VALLEY GENERAL HOSPITAL,93230,HCPCS 10120,"Incision And Removal Of Foreign Body, Subcutan...",$162.30
3,CENTRAL VALLEY GENERAL HOSPITAL,93230,HCPCS 10140,Surgical Drainage Of Hematoma Or Seroma,$753.30
4,CENTRAL VALLEY GENERAL HOSPITAL,93230,HCPCS 10160,Image-Guided Puncture And Drainage (Aspiration...,$83.57
...,...,...,...,...,...
1245,KINDRED HOSPITAL BALDWIN PARK,91706,HCPCS 86803,Confirmation Test For Hepatitis C Antibody,$63.36
1246,KINDRED HOSPITAL BALDWIN PARK,91706,HCPCS 10061,Drainage Of Skin Abscess,$649.80
1247,KINDRED HOSPITAL BALDWIN PARK,91706,HCPCS 10140,Surgical Drainage Of Hematoma Or Seroma,"$1,527.84"
1248,KINDRED HOSPITAL BALDWIN PARK,91706,HCPCS 10060,Drainage Of Abscess,$541.44


In [141]:
pricing_df.to_excel('/Users/amanchopra/Documents/Work/Aarete/Provider Transparency Project/TH_pricing_info.xlsx', index=False)